In [1]:
import pickle
import pandas as pd

In [2]:
import re
import time
import datetime

In [3]:
# !pip install spacy
# !python3 -m spacy download en_core_web_sm

In [4]:
from string import punctuation
import spacy

In [5]:
import random

In [35]:
from p05_nlp import vectorize_tfidf
from p05_nlp import reduce_dim_lsa
from p05_nlp import reduce_dim_lda
from p05_nlp import reduce_dim_nmf
from p05_nlp import display_topics

In [6]:
with open("df_cult.pickle", "rb") as read_file:
    df_cult = pickle.load(read_file)

In [7]:
len(df_cult)

1678

In [8]:
df_cult.columns

Index(['title', 'url', 'year', 'director', 'wiki_page', 'poster_url',
       'imdb_url'],
      dtype='object')

In [9]:
with open("simple_scrape_dict.pickle", "rb") as read_file:
    reviews_dict = pickle.load(read_file)

In [10]:
len(reviews_dict)

1670

In [11]:
reviews_dict[('The Killing', 'https://www.imdb.com/title/tt0049406/')]

['\'The Killing\' has been overshadowed by Stanley Kubrick\'s subsequent better known and better made movie masterpieces. Films like \'Dr. Strangelove\', \'2001\' and \'A Clockwork Orange\' are much more flamboyant and intellectually exciting than this early hard boiled crime thriller, but for my money it is still one of his most entertaining movies, and in its own modest way just as brilliant as his more talked about films. \'The Killing\' is still one of the greatest crime thrillers ever made, and one which influenced many film makers working in this genre, not the least of which Quentin Tarantino, who obviously worships this picture, and used its innovative structure as major inspiration for \'Pulp Fiction\'.  Kubrick wrote \'The Killing\'s script as well as directing, but made the smart move of asking "the Dime Store Dostoevski" Jim Thompson, author of pulp classics like \'The Killer Inside Me\' and \'The Getaway\' to supply the fresh and memorable dialogue. Sterling Hayden, who la

In [12]:
reviews_joined = {key:" ".join(val) for key, val in reviews_dict.items()}

In [13]:
len(reviews_joined)

1670

In [14]:
reviews_joined[('The Killing', 'https://www.imdb.com/title/tt0049406/')]

'\'The Killing\' has been overshadowed by Stanley Kubrick\'s subsequent better known and better made movie masterpieces. Films like \'Dr. Strangelove\', \'2001\' and \'A Clockwork Orange\' are much more flamboyant and intellectually exciting than this early hard boiled crime thriller, but for my money it is still one of his most entertaining movies, and in its own modest way just as brilliant as his more talked about films. \'The Killing\' is still one of the greatest crime thrillers ever made, and one which influenced many film makers working in this genre, not the least of which Quentin Tarantino, who obviously worships this picture, and used its innovative structure as major inspiration for \'Pulp Fiction\'.  Kubrick wrote \'The Killing\'s script as well as directing, but made the smart move of asking "the Dime Store Dostoevski" Jim Thompson, author of pulp classics like \'The Killer Inside Me\' and \'The Getaway\' to supply the fresh and memorable dialogue. Sterling Hayden, who lat

In [15]:
titles = []
imdb_urls = []
reviews_lists = []
for key, val in reviews_joined.items():
    titles.append(key[0])
    imdb_urls.append(key[1])
    reviews_lists.append(val)

reviews_df = pd.DataFrame.from_dict({'title':titles, 'imdb_url':imdb_urls, 'reviews_list':reviews_lists})

In [16]:
reviews_df

,title,imdb_url,reviews_list
0,200 Motels,https://www.imdb.com/title/tt0066732/,"There is no film quite like 200 Motels, but a ..."
1,2001: A Space Odyssey,https://www.imdb.com/title/tt0062622/,"A stand-alone monument in cinema history, Stan..."
2,37°2 le Matin (Betty Blue),https://www.imdb.com/title/tt0090563/,The Betty of the title is like a shooting star...
3,42nd Street,https://www.imdb.com/title/tt0024034/,Most of the negative comments posted below see...
4,The 7th Voyage of Sinbad,https://www.imdb.com/title/tt0051337/,"Arguably, Harryhausen's finest moment. I can't..."
5,9 Songs,https://www.imdb.com/title/tt0411705/,How does a modern man recall his last 'love' r...
6,300,https://www.imdb.com/title/tt0416449/,This film isn't for all people. That's to say ...
7,A Clockwork Orange,https://www.imdb.com/title/tt0066921/,Few films are as sensational or infamous as St...
8,À Bout de Souffle (Breathless),https://www.imdb.com/title/tt0053472/,I don't blame those who state that they do not...
9,À Meia-Noite Levarei Sua Alma (At Midnight I'l...,https://www.imdb.com/title/tt0059440/,"I love the idea of Coffin Joe (""Joseph The Gra..."


In [17]:
df_all = reviews_df.merge(df_cult, how='left', on=['title','imdb_url'])

In [18]:
df_all.columns

Index(['title', 'imdb_url', 'reviews_list', 'url', 'year', 'director',
       'wiki_page', 'poster_url'],
      dtype='object')

In [19]:
df_all = df_all[['title','year','director','reviews_list','imdb_url','wiki_page','url','poster_url']]

In [20]:
df_all.head()

,title,year,director,reviews_list,imdb_url,wiki_page,url,poster_url
0,200 Motels,1971,Tony Palmer,"There is no film quite like 200 Motels, but a ...",https://www.imdb.com/title/tt0066732/,https://en.wikipedia.org/wiki/200_Motels,/wiki/200_Motels,upload.wikimedia.org/wikipedia/en/thumb/8/85/2...
1,2001: A Space Odyssey,1968,Stanley Kubrick,"A stand-alone monument in cinema history, Stan...",https://www.imdb.com/title/tt0062622/,https://en.wikipedia.org/wiki/2001:_A_Space_Od...,/wiki/2001:_A_Space_Odyssey_(film),upload.wikimedia.org/wikipedia/en/thumb/9/94/S...
2,37°2 le Matin (Betty Blue),1986,Jean-Jacques Beineix,The Betty of the title is like a shooting star...,https://www.imdb.com/title/tt0090563/,https://en.wikipedia.org/wiki/Betty_Blue,/wiki/Betty_Blue,upload.wikimedia.org/wikipedia/en/thumb/9/9d/B...
3,42nd Street,1933,Lloyd Bacon,Most of the negative comments posted below see...,https://www.imdb.com/title/tt0024034/,https://en.wikipedia.org/wiki/42nd_Street_(film),/wiki/42nd_Street_(film),upload.wikimedia.org/wikipedia/en/thumb/8/8b/F...
4,The 7th Voyage of Sinbad,1957,Nathan H. Juran,"Arguably, Harryhausen's finest moment. I can't...",https://www.imdb.com/title/tt0051337/,https://en.wikipedia.org/wiki/The_7th_Voyage_o...,/wiki/The_7th_Voyage_of_Sinbad,upload.wikimedia.org/wikipedia/en/thumb/a/a6/S...


In [21]:
reviews = df_all['reviews_list'].copy()

In [22]:
clean_test = random.choices(reviews, k=5)
clean_test

['This film is not a feature film.  For a start, it is not feature length, also, it is not shot on film.  More importantly, it does not have what feature films have these days: star actors, special effects, exotic locations, explosions.  Instead, seeing B.W.P. is seeing something else that a cinema can be: a place where people can share an intimate experience created by a few people on a tight budget.  I would be glad of its success if only for that reason.The first section of the film appears at first to be amateurish and slow. In fact, it is very deft, and very efficient at what it does.  It tells the audience everything it needs to know about the characters and situation, and nothing more.  Also, it gets the audience into the habit of viewing the film\'s format: alternating between black and white (very grainy and poorly focussed) film, and the washed out colours of shaky pixilated video.  The film makers managed to set up a rationale for why the film is so cheaply made.  Three peop

# Test text cleaning operation on random sample clean_test

In [23]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
# remove backslashes, "'s", and "x85"
backslash = re.compile('\[\\]')
x85 = re.compile('(\\x85)')
x96 = re.compile('(\\x96)')
posessive = re.compile("\'s")

# remove numbers
alphanumeric = lambda x: re.sub('\w*\d\w*|&gt;', '', x)
# remove punctuation, convert to lowercase
punc_lower = lambda x: re.sub('[%s]' % re.escape(punctuation), ' ', x.lower())

clean_test = pd.Series(clean_test)
clean_test = clean_test.map(lambda x: re.sub(backslash, ' ', x))
clean_test = clean_test.map(lambda x: re.sub(x85, '', x))
clean_test = clean_test.map(lambda x: re.sub(x96, '', x))
clean_test = clean_test.map(lambda x: re.sub(posessive, '', x))
clean_test = clean_test.map(alphanumeric)
clean_test = clean_test.map(punc_lower)

In [25]:
clean_test[0]

'this film is not a feature film   for a start  it is not feature length  also  it is not shot on film   more importantly  it does not have what feature films have these days  star actors  special effects  exotic locations  explosions   instead  seeing b w p  is seeing something else that a cinema can be  a place where people can share an intimate experience created by a few people on a tight budget   i would be glad of its success if only for that reason the first section of the film appears at first to be amateurish and slow  in fact  it is very deft  and very efficient at what it does   it tells the audience everything it needs to know about the characters and situation  and nothing more   also  it gets the audience into the habit of viewing the film format  alternating between black and white  very grainy and poorly focussed  film  and the washed out colours of shaky pixilated video   the film makers managed to set up a rationale for why the film is so cheaply made   three people h

In [26]:
tokenizer = spacy.load('en_core_web_sm')

def tokenize_lemma(text):
        
    text_obj = tokenizer(text, disable=['parser', 'ner'])
    
    text_lemma = ' '.join([token.lemma_ for token in text_obj if not token.is_stop])
  
    return text_lemma

In [27]:
lemmatize = lambda x: tokenize_lemma(x)
clean_test = clean_test.map(lemmatize)

In [28]:
clean_test[0]

'film feature film    start   feature length     shoot film    importantly   feature film day   star actor   special effect   exotic location   explosion    instead   see b w p   see cinema   place people share intimate experience create people tight budget    glad success reason section film appear amateurish slow   fact   deft   efficient    tell audience need know character situation        get audience habit view film format   alternate black white   grainy poorly focussed   film   washed colour shaky pixilate video    film maker manage set rationale film cheaply    people hike wood day shoot documentary   borrow equipment   habit videoe hell     carry tripod   steadicam   dolly   large lighting rig   like   light raw daylight   single light fix camera   illuminate foot lens    film create excuse cheap    intelligent acting script excellent    cast actor presumably play pretty   convincingly naturalistic   likeable dislikeable    slow route hysteria document    simply have characte

## test topic modeling steps (to use in next notebook)

In [36]:
# vectorize_tfidf
clean_test_sparse_matrix, clean_test_feature_names = vectorize_tfidf(clean_test)


    ability      able  absolutely     abuse    accept  accompany  accomplish  \
0  0.000000  0.009982    0.014972  0.006012  0.006012   0.006012    0.000000   
1  0.000000  0.000000    0.000000  0.000000  0.000000   0.000000    0.000000   
2  0.009894  0.000000    0.024640  0.000000  0.000000   0.000000    0.000000   
3  0.000000  0.007073    0.000000  0.000000  0.008520   0.000000    0.008520   
4  0.015316  0.012714    0.006357  0.007658  0.000000   0.007658    0.007658   

    achieve      acid       act  ...    writer   writing     wrong      year  \
0  0.006012  0.000000  0.012595  ...  0.004198  0.000000  0.016794  0.028408   
1  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.113062  0.095627   
2  0.009894  0.009894  0.055274  ...  0.006909  0.000000  0.000000  0.011688   
3  0.000000  0.000000  0.011899  ...  0.023799  0.017041  0.005950  0.055355   
4  0.000000  0.007658  0.021391  ...  0.085562  0.038291  0.026738  0.022615   

       yell       yep       yes     yo

In [39]:
# reduce_dim_lsa
clean_test_lsa, lsa_components = reduce_dim_lsa(clean_test_sparse_matrix, 3)
# display_topics

In [40]:
display_topics(lsa_components, clean_test_feature_names, 10)


Topic  0
film, movie, like, time, bad, watch, see, good, scene, think

Topic  1
faith, trash, definitive, heaven, signifying, favorite, major, obscene, eyed, primarily

Topic  2
surf, surfing, surfer, big, bear, scene, wave, grow, party, beach


# Clean reviews text i.e. deep copy of df_all.reviews_text

In [60]:
# remove backslashes, "'s", and "x85"
backslash = re.compile('\[\\]')
x85 = re.compile('(\\x85)')
x96 = re.compile('(\\x96)')
posessive = re.compile("\'s")

# remove numbers
alphanumeric = lambda x: re.sub('\w*\d\w*|&gt;', '', x)
# remove punctuation, convert to lowercase
punc_lower = lambda x: re.sub('[%s]' % re.escape(punctuation), ' ', x.lower())

# replace all of (custom stop words list) with empty string
custom_stop = ['woody','allen','movie','film','watch','character','see','story','animate']
# ['going','just','let','minus','professor','audience','play','okay','print','mit','respect','gonna','ok','thats','im','right','tes']
rem_stop = lambda x: re.sub("|".join(custom_stop), "", x)


reviews = pd.Series(reviews)
reviews = reviews.map(lambda x: re.sub(backslash, ' ', x))
reviews = reviews.map(lambda x: re.sub(x85, '', x))
reviews = reviews.map(lambda x: re.sub(x96, '', x))
reviews = reviews.map(lambda x: re.sub(posessive, '', x))
reviews = reviews.map(alphanumeric)
reviews = reviews.map(punc_lower)
# reviews = reviews.map(rem_stop)

In [61]:
tokenizer = spacy.load('en_core_web_sm')

def tokenize_lemma(text):
        
    text_obj = tokenizer(text, disable=['parser']) #, 'ner'])
    
    text_lemma = ' '.join([token.lemma_ for token in text_obj if not token.is_stop])
  
    return text_lemma

In [62]:
lemmatize = lambda x: tokenize_lemma(x)
reviews = reviews.map(lemmatize)

In [63]:
reviews[325]

'love      grow    condorman think fond memory     special effect good        decade old     think effect adequate        suppose realistic     comedy action element     provide window people worry early eighty     cold war espionage major      overall     charming feel good      classic element     good verse evil boy meet girl     quality family    recommend    young heart     good james bond      g     rating     look like     condorman      cool gadget     world wide location     sexy femme fatale     creepy russain villain     good     cool car boat chase sequence      mind kid      humor level elementary     nonetheless funny     act cartoonish     regardlessly enjoyable     look fun     fast pace adventure    check      grow       time star michael crawford persona frank spencer add american accent cartoon artist help friend work cia job merely excuse fun joke cool action sequence     bit porsche excellent     think james bond kid bit romance worry     great    kid     okay adul

# Add cleaned reviews to df_all

In [64]:
df_all['reviews_clean'] = reviews

In [65]:
df_all

,title,year,director,reviews_list,imdb_url,wiki_page,url,poster_url,reviews_clean
0,200 Motels,1971,Tony Palmer,"There is no film quite like 200 Motels, but a ...",https://www.imdb.com/title/tt0066732/,https://en.wikipedia.org/wiki/200_Motels,/wiki/200_Motels,upload.wikimedia.org/wikipedia/en/thumb/8/85/2...,like motel lot strange appearance ...
1,2001: A Space Odyssey,1968,Stanley Kubrick,"A stand-alone monument in cinema history, Stan...",https://www.imdb.com/title/tt0062622/,https://en.wikipedia.org/wiki/2001:_A_Space_Od...,/wiki/2001:_A_Space_Odyssey_(film),upload.wikimedia.org/wikipedia/en/thumb/9/94/S...,stand monument cinema hi stanley kubrick m...
2,37°2 le Matin (Betty Blue),1986,Jean-Jacques Beineix,The Betty of the title is like a shooting star...,https://www.imdb.com/title/tt0090563/,https://en.wikipedia.org/wiki/Betty_Blue,/wiki/Betty_Blue,upload.wikimedia.org/wikipedia/en/thumb/9/9d/B...,betty title like shoot star run hot bright...
3,42nd Street,1933,Lloyd Bacon,Most of the negative comments posted below see...,https://www.imdb.com/title/tt0024034/,https://en.wikipedia.org/wiki/42nd_Street_(film),/wiki/42nd_Street_(film),upload.wikimedia.org/wikipedia/en/thumb/8/8b/F...,negative comment post people don t like musica...
4,The 7th Voyage of Sinbad,1957,Nathan H. Juran,"Arguably, Harryhausen's finest moment. I can't...",https://www.imdb.com/title/tt0051337/,https://en.wikipedia.org/wiki/The_7th_Voyage_o...,/wiki/The_7th_Voyage_of_Sinbad,upload.wikimedia.org/wikipedia/en/thumb/a/a6/S...,arguably harryhausen fine moment t hea...
5,9 Songs,2004,Michael Winterbottom,How does a modern man recall his last 'love' r...,https://www.imdb.com/title/tt0411705/,https://en.wikipedia.org/wiki/9_Songs,/wiki/9_Songs,upload.wikimedia.org/wikipedia/en/thumb/2/26/9...,modern man recall love relationship ...
6,300,2006,Zack Snyder,This film isn't for all people. That's to say ...,https://www.imdb.com/title/tt0416449/,https://en.wikipedia.org/wiki/300_(film),/wiki/300_(film),upload.wikimedia.org/wikipedia/en/thumb/e/e7/C...,isn t people lot general course ...
7,A Clockwork Orange,1971,Stanley Kubrick,Few films are as sensational or infamous as St...,https://www.imdb.com/title/tt0066921/,https://en.wikipedia.org/wiki/A_Clockwork_Oran...,/wiki/A_Clockwork_Orange_(film),upload.wikimedia.org/wikipedia/en/thumb/7/73/A...,sensational infamous stanley kubrick cl...
8,À Bout de Souffle (Breathless),1960,Jean-Luc Godard,I don't blame those who state that they do not...,https://www.imdb.com/title/tt0053472/,https://en.wikipedia.org/wiki/Breathless_(1960...,/wiki/Breathless_(1960_film),upload.wikimedia.org/wikipedia/en/3/3f/%C3%80_...,don t blame state understand superlati...
9,À Meia-Noite Levarei Sua Alma (At Midnight I'l...,1964,José Mojica Marins,"I love the idea of Coffin Joe (""Joseph The Gra...",https://www.imdb.com/title/tt0059440/,https://en.wikipedia.org/wiki/At_Midnight_I%27...,/wiki/At_Midnight_I%27ll_Take_Your_Soul,upload.wikimedia.org/wikipedia/en/thumb/9/99/Q...,love idea coffin joe joseph grave l...


In [66]:
with open("df_all_cleaned.pickle", "wb") as write_file:
    pickle.dump(df_all, write_file)

In [ ]:
# def standardize(df):
#     # replace '\n' with whitespace
#     backslash = lambda x: re.sub('\\|\\x85', '', x)
    
#     # remove numbers
#     alphanumeric = lambda x: re.sub('\w*\d\w*|&gt;', '', x)
#     # remove punctuation, convert to lowercase
#     punc_lower = lambda x: re.sub('[%s]' % re.escape(punctuation), '', x.lower())
    
#     lemmatize = lambda x: tokenize_lemma(x)
    
#     # replace all "taylor's" with "taylor"
#     df['text'] = df['text'].str.replace("taylor's", "taylor", case = False)
#     df['text'] = df['text'].str.replace("transformation", "transform", case = False)
    
#     # replace all of (custom stop words list) with empty string
#     custom_stop = ['going','just','let','minus','professor','audience','play','okay','print','mit','respect','gonna','ok','thats','im','right','tes']
#     rem_stop = lambda x: re.sub("|".join(custom_stop), "", x)
    
#     df['text'] = df.text.map(newline).map(alphanumeric).map(punc_lower)
#     df['text'] = df['text'].str.replace("laplace transform", "laplace_transform", case = False)
#     df['text'] = df['text'].str.replace("partial derivative", "partial_derivative", case = False)
#     df['text'] = df['text'].str.replace("power series expansion", "power_series_expansion", case = False)
#     df['text'] = df.text.map(rem_stop)
#     df['text'] = df.text.map(lemmatize)
    
#     return df